This version use entropy/information of each word in allowed guesses to rank and choose which will be guess in next step <br />
Entropy shows the expected information given by each word, is used to reduce the uncertainty of possible answer then easily minimum the score (number of tries) which is less than 6 <br />
Note: Guesses could be chosen from reduced possible answers list(hard mode) or allowed guesses list, it depend on purpose: choose answer or get information ( green,yellow,..)


#### Algorithm
list=allowed_lst <br />
while feedback != all 2's <br />
choose Max entropy from list<br />
list= feedback + guess <br />

entropy= E(I)=sum(p.I)=sum(p.log2(1/p)) <br />
Compute p of each pattern which this word may have <br />
p=pattern made by remain possible answers match with guess / list <br />

save best choice at each guess with each pattern <br />



In [2]:
with open('allowed_guesses.txt','r') as file:
    allowed_guesses=[]
    for i in file:
        allowed_guesses.append(i[:5])
with open("real_possible_answers.txt","r") as file:
        real_possible_answers=[]
        for i in file:
            real_possible_answers.append(i[:5])
import random
import time
from math import *

In [3]:
def get_feedback(guess:str,answer:str) -> list: 
    """
    get_feeback
    ----------
    guess: Five-letter guess.
    answer: Five-letter correct answer.
    Returns: Feedback contains 05 elements, which can be 0, 1, or 2, denoting a feedback pattern
    """
    # convert string to list
    answer = list(answer)
    guess = list(guess)
    
    #initialize
    feedback = ['']*5
    
    # isolate correctly placed letters
    for i in range(5):
        if guess[i] == answer[i]:
            feedback[i] = 2
            answer[i] = ''
            guess[i] = ''

    for i in range(5):
        if guess[i] == '': continue
        try:
            index=answer.index(guess[i])
            feedback[i] = 1
            answer[index] = ''
            guess[i] = ''
        except:
            feedback[i]=0
    return feedback

get_feedback('speed','crepe')

[0, 1, 2, 1, 0]

In [4]:
def reduce_list(guess:str,feedback:list,possible_answers:list) ->list:
    '''This function return the reduced allowed word list - possible answers list which have real answer
    
    guess: the word we guessed in this step
    feedback: the pattern given by game
    word_list: allowed list in the first step and the reduced list in the next steps
    '''
    return [word for word in possible_answers if get_feedback(guess,word)==feedback]
len(reduce_list('slate',[1,0,1,2,0],allowed_guesses))



74

In [9]:
def convert_ternary(feedback):
    """
    Parameters
    ----------
    Feedback: Contains 05 elements, which can be 0, 1, or 2, denoting a feedback pattern.
    Returns
    -------
    Number: Integer base 10 representation of pattern.
    """
    return sum([feedback[i]*3**(4-i) for i in range(5)])
convert_ternary([1,2,0,0,2])

137

In [5]:
def entropy(guess:str, possible_answers:list):
    '''Function compute the entropy of each word which could be chosen in hard mode \n
    Return value of entropy(bits), dictionary has key= pattern and value= reduced possible answers list \n  
    Detail: It computes expected value of probability distribution of possible patterns, \n 
    based on possible answer(reduced list) or allowed list at initial \n
    pattern=guess + word in possible answers \n
    p= number of specific pattern / total patterns of word_list'''
    Entropy=0
    total = len(possible_answers) 
    PD_patterns = {} #probability distribution of patterns
    for word in possible_answers: 
        feedback = get_feedback(guess,word)
        fb=convert_ternary(feedback)
        # PD_patterns[fb] = PD_patterns.get(fb,[]) +[word] 
        PD_patterns[fb]=PD_patterns.get(fb,0)+1/total
    for prob in PD_patterns.values():
        # Numberofpattern=len(lst)
        # prob=Numberofpattern/total
        # prob=lst
        Entropy+=-(prob)*(log(prob)/log(2))
    # print(PD_patterns)
    return Entropy
# entropy('tares',allowed_guesses)


In [6]:
def entropy_dict(possible_answers):
    '''Function compute entropy of each word in list
    Return the rank of possible answer (result of entropy function) based value of entropy
    It may take less then 10 mins to compute for the first list - first guess'''
    ranker=[]
    for guess in possible_answers:
        ranker.append((guess,entropy(guess,possible_answers)))
    ranker.sort(key = lambda t: t[1], reverse = True)
    return ranker
    # pair=(possible_answers[0],entropy(possible_answers[0],possible_answers))
    # ranker = [pair]
    # for guess in possible_answers:
    #     new_pair=(guess,entropy(guess,possible_answers))
    #     change=0
    #     for i,pair in enumerate(ranker):
    #         if new_pair[1][0]>=pair[1][0]:
    #             ranker=ranker[:i]+[new_pair]+ranker[i:]
    #             change=1
    #             break
    #     if change==0:
    #         ranker+=[new_pair]
    # return ranker
# print([(word,entropy(word,allowed_guesses[:100])) for word in allowed_guesses[:100]])
# print(allowed_guesses[:100])
#1m21.7
#1m20.6s my own
# 11.38.1s all words
# length=len(allowed_guesses)//3
# def en(list):
#     a=entropy_dict(list[:length])
#     b=entropy_dict(list[length:length*2])
#     c=entropy_dict(list[length*2:])
#     return a+b+c
# print(a)
# entropy_dict(allowed_guesses[4000:8000])
# # print(b)
# c=entropy_dict(allowed_guesses[8000:])
# print(c)
# 2m44s
#2m43.9s done
# def entropy_dict(possible_answers): #8m11.6s
#     a=entropy_subdict(allowed_guesses[:4000],allowed_guesses)
#     b=entropy_subdict(allowed_guesses[4000:8000],allowed_guesses)
#     c=entropy_subdict(allowed_guesses[8000:],allowed_guesses)
#     return a+b+c
# ranker=entropy_dict(allowed_guesses[:3])
# ranker.sort(key = lambda t: t[1][0], reverse = True)
# print(ranker)


In [ ]:
#saving first ranker of entropy
import json
with open('firstguesses_hardmode.json','w') as f:
    json.dump(entropy_dict(allowed_guesses),f)
with open('firstguesses_hardmode.json','r') as f:
    file=json.load(f)
print(file[0])
    

: 

[('tares', 6.195594730245377),
 ('lares', 6.151313589324651),
 ('rales', 6.115942447723608),
 ('rates', 6.097615309189668),
 ('teras', 6.078258349905183),
 ('nares', 6.068444540347897),
 ('soare', 6.062302776061357),
 ('tales', 6.0563053370969895),
 ('reais', 6.051229434211681),
 ('tears', 6.033781875729947),
 ('arles', 6.031334630198202),
 ('tores', 6.019304195305773),
 ('salet', 6.017847926120458),
 ('aeros', 6.0140608641637145),
 ('dares', 6.011187647555291),
 ('reals', 6.000797839298542),
 ('saner', 6.000576805791645),
 ('lears', 5.990079707202211),
 ('lores', 5.977642150779597),
 ('serai', 5.974867726286669),
 ('lanes', 5.972511722164216),
 ('laers', 5.9702900415510864),
 ('pares', 5.968213362998895),
 ('cares', 5.96714476106444),
 ('tires', 5.964383595082227),
...
 ('morts', 5.392590605517464),
 ('leres', 5.392100150679694),
 ('peart', 5.392010978047047),
 ('lunas', 5.391224654535519),
 ...]

Because winrate is not 100% so Buidling tree is not used, instead by saving first list (takes too long time) and computing reduced list,entropy simultaneously (too long time). </br>
if 100%, save all best guess of each (pattern + previous guess)

In [7]:
def solution(answer,input=False):
    '''This function return the number of step in this play\n
    answer: The secret word of the game
    input : True is feedback inputted from terminal and vice versa
    '''
    k=0
    guess='tares'
    word_list=allowed_guesses
    while True:
        k+=1
        # print(guess)
        if input:
            feedback=[int(i) for i in str(input())]
        else:
            feedback=get_feedback(guess,answer)
        if feedback==[2]*5:
            break
        word_list=reduce_list(guess,feedback,word_list)
        guess=entropy_dict(word_list)[0][0]
    return k

In [10]:
def TestModel(solution,test_list:list,RANDOM=False) -> tuple:
    '''
    Parameter
    ----------
    solution: The function return the number of step to guess a specific answer \n
    test_list: The list of answers for testing
    RANDOM: False, answer is chosen randomly and True, answer is chosen sequentially from the test_list
    ----------
    Return: The bar chart with x(number of guesses needed) and y (number of plays having x guesses) \n
                  The tupple of winrate and average score'''
    import random
    import time
    t1=time.time()
    #Compute some vital factor: number of plays having x guesses, win rate, average score of 2,3k plays   
    xMax=20 # may be posituve infinity number
    yMax=0
    lst=[0]*xMax
    N=len(test_list)# list contains number of plays having x guesses
    for word in test_list:
        if not RANDOM:
            answer=word
        elif RANDOM:
            answer=random.choice(test_list)
        NumberOfGuessesNeeded=solution(answer) 
        lst[NumberOfGuessesNeeded]=lst[NumberOfGuessesNeeded]+1
    winrate=sum(lst[1:7])/N*100
    average=sum([i*lst[i] for i in range(1,xMax)]) / N

    #VISUALIZATION
    import matplotlib.pyplot as plt
    for i in range(1,xMax):
        if lst[i] >=yMax: # because yMax always in (1,6)
            yMax=lst[i]
        if lst[i]==0 and i>6:
            xMax=i
            break
    yMax=(yMax//100+2)*100
    x=[str(i) for i in range(1,xMax)]
    y=[i for i in lst[1:xMax]]
    plt.ylim(0,yMax)
    plt.grid(axis='y',linestyle='--')
    plt.xlabel('Number of guesses needed')
    plt.ylabel('Number of plays having x guesses')
    plt.title('TEST PERFORMANCE')
    plt.bar(x,y, fc="#CCD6A6", ec="black")
    for i in range(len(x)):
        plt.text(i, y[i], y[i], ha="center", va="bottom")
    t2=time.time()
    time=t2-t1
    plt.text(xMax,yMax/2, f'Win Rate: {winrate:.3f}%\nAverage Score: {average:.3f}\nTime: {time:.3f}s', fontsize = 20,
		bbox = dict(facecolor = '#CCD6A6', alpha = 0.7))
    plt.show()
    return winrate,average

TestModel(solution,real_possible_answers)
        
